In [2]:
import os
import pandas as pd
import pickle


#exp_name = 'preprocessing_type'
#experiments = ["adam", "other", "rmsprop", "sgd"]
pending_logs_path = os.path.join('mac_logs', 'run_pending_experiments')
peephole_logs_path = os.path.join('mac_logs', 'peephole_experiments')

def load_gridsearch_results(logs_path):
    grid_search_results = []
    results_path = os.path.join(logs_path, 'gridsearch_results.pkl')
    with open(results_path, 'rb') as f:
        partial_results = pickle.load(f)
    grid_search_results.extend(partial_results)
    return grid_search_results

def add_peephole_param(results):
    for res in results:
        if "model-rnn_cell_params-peephole_connections" not in res:
            res["model-rnn_cell_params-peephole_connections"] = False
    return results

pending_res = load_gridsearch_results(pending_logs_path)
pending_res = add_peephole_param(pending_res)
pending_keys = pending_res[0].keys()
peephole_res = load_gridsearch_results(peephole_logs_path)
peephole_keys = peephole_res[0].keys()




print('pending keys:', pending_keys)
print("peephole keys:", peephole_keys)
print(type(peephole_res[0]["model-rnn_cell_params-peephole_connections"]))  
print(type(pending_res[0]["model-rnn_cell_params-peephole_connections"]))

grid_search_results = pending_res + peephole_res

pending keys: dict_keys(['model-cell_type', 'model-rnn_cell_params-units', 'metrics_per_fold', 'model-rnn_cell_params-peephole_connections'])
peephole keys: dict_keys(['model-cell_type', 'model-rnn_cell_params-units', 'model-rnn_cell_params-peephole_connections', 'metrics_per_fold'])
<class 'bool'>
<class 'bool'>


In [3]:
print(grid_search_results)

[{'model-cell_type': 'lstm', 'model-rnn_cell_params-units': 10, 'metrics_per_fold': [{'mean_squared_error': {'scaled': <optimization.data_classes.MetricResults object at 0x000002D5B485B650>, 'unscaled': <optimization.data_classes.MetricResults object at 0x000002D5DC077EF0>}, 'root_mean_squared_error': {'scaled': <optimization.data_classes.MetricResults object at 0x000002D5DC09C920>, 'unscaled': <optimization.data_classes.MetricResults object at 0x000002D5DC09D310>}, 'mean_absolute_error': {'scaled': <optimization.data_classes.MetricResults object at 0x000002D5DC09DD00>, 'unscaled': <optimization.data_classes.MetricResults object at 0x000002D5DC09E6F0>}}, {'mean_squared_error': {'scaled': <optimization.data_classes.MetricResults object at 0x000002D5DC09F110>, 'unscaled': <optimization.data_classes.MetricResults object at 0x000002D5DC09FB00>}, 'root_mean_squared_error': {'scaled': <optimization.data_classes.MetricResults object at 0x000002D5DC0A8530>, 'unscaled': <optimization.data_class

In [4]:
import pickle

with open(os.path.join(pending_logs_path, 'basic_parameters.pickle'), 'rb') as f:
    basic_parameters = pickle.load(f)
print(basic_parameters)

{'dataset': {'data_path': '/Users/maderajan/Programming/LSTM-crystal-growth/python/data', 'scaler': StandardScaler(), 'preprocessing_function': <function lag_features at 0x000002D5DDE6D760>, 'shuffle': True, 'final_testing': False, 'test_size': 0.0, 'outputs_lag': 3, 'inputs_lag': 0, 'window_size': 3, 'return_sequences': True, 'seed': 42, 'scaler_partitions': [[0, 1], [2, 3, 4, 5, 6], [7]], 'scaler_kwargs': {}, 'k_folds': 10}, 'dataset_class': <class 'data_processing.dataset.CrystalDataset'>, 'model': {'bidirectional': False, 'cell_type': 'gru', 'rnn_cell_params': {'units': 20}, 'num_layers': 1, 'rnn_layer_params': {'return_sequences': True}, 'dilation_base': None, 'residual_block_size': None, 'smyl_std': False, 'smyl_residual': False}, 'training': {'batch_size': 32, 'epochs': 200, 'optimizer': 'rmsprop', 'early_stopping': False, 'model_checkpoint': True, 'verbose': 0, 'shuffle': True, 'optimizer_params': {'learning_rate': 0.0001, 'momentum': 0.99, 'clipnorm': 5.0}, 'early_stopping_par

In [5]:
print(len(grid_search_results))
for exp in grid_search_results:
    for key in exp.keys():
        if key != 'metrics_per_fold':
            print(f"{key}: {exp[key]}")
    #print(f"epochs: {exp["training-epochs"]}")
    folds = exp["metrics_per_fold"]
    for i, fold in enumerate(folds):
        print(f"\tFold {i+1}:")
        for metric, values in fold.items():
            scaled_val = values["scaled"]
            unscaled_val = values["unscaled"]
            print(f"\t\t{metric}: {scaled_val.metric:.6f} ({unscaled_val.metric:.6f})\n\t\t\tper_sample s (u):\n\t\t\t\t {scaled_val.metric_per_sample}\n\t\t\t\t({unscaled_val.metric_per_sample})")
    print()
    print("######################################")

15
model-cell_type: lstm
model-rnn_cell_params-units: 10
model-rnn_cell_params-peephole_connections: False
	Fold 1:
		mean_squared_error: 0.000106 (5.296237)
			per_sample s (u):
				 [9.125182e-05, 4.231633e-05, 8.87686e-05, 4.2179974e-05, 0.00020661169, 0.00011867708, 4.2927706e-05, 0.0001224174, 3.4944835e-05, 5.524829e-05, 0.00017903041, 0.00020863368, 3.8090227e-05, 4.4020806e-05, 0.00012289103, 0.00012317367, 6.0795268e-05, 0.00012417437, 4.06193e-05, 6.135669e-05, 3.933243e-05, 5.2093066e-05, 4.18934e-05, 5.648455e-05, 4.451061e-05, 0.00027509412, 0.00021307297, 0.00013960114, 0.0002180235, 0.00020796084, 0.00017757004, 4.2507218e-05, 0.000119945034, 3.5660243e-05, 3.8160368e-05, 0.00022035399, 3.771256e-05, 3.6368154e-05, 0.00022017116, 4.4957033e-05, 0.00024652178, 4.387791e-05, 0.00020467943, 0.00012926995, 4.168197e-05]
				([2.8772936, 1.2301403, 2.7644336, 1.2326472, 11.484026, 4.293529, 1.2316664, 4.887609, 2.4812148, 1.5032157, 10.691336, 11.3369665, 4.1352305, 4.8278127

1. 
   - For each experiment, get all samples. 
   - The samples are ordered for each experiment in the same way. 

In [11]:
from sklearn.preprocessing import FunctionTransformer
from types import FunctionType
import re

def get_exp_name(exp):
    name = ""
    for i, key in enumerate(exp.keys()):
        if key != 'metrics_per_fold':
            if i > 0:
                name += " "
            if isinstance(exp[key], str):
                name += exp[key]
            elif isinstance(exp[key], FunctionTransformer):
                return exp[key].func.__name__
            elif isinstance(exp[key], bool):
                name += key if exp[key] else "not "+ key
            elif isinstance(exp[key], FunctionType):
                name += exp[key].__qualname__
            else:
                key_parts = key.split("-")
                last_part = key_parts[-1]
                key_words = last_part.split("_")
                shortcut = "".join([word[0] for word in key_words]) if len(key_words) > 1 else key_words[0]
                name += shortcut + "=" + str(exp[key])

    #name = name.replace("dataset-", "").replace("_", " ").replace("-", " ")
    name = name.capitalize()
    return name

def is_gru(exp):
    if 'model-cell_type' in exp and exp['model-cell_type'] == 'gru':
        return True
    return False

exp_samples = {}
for i, exp in enumerate(grid_search_results):
    if is_gru(exp):
        continue
    exp_name = get_exp_name(exp)
    exp_samples[exp_name] = []
    folds = exp["metrics_per_fold"]
    for fold in folds:
        exp_samples[exp_name].extend(fold["root_mean_squared_error"]["unscaled"].metric_per_sample)
    #print(len(exp_samples[exp_name]))

#for exp_name, samples in exp_samples.items():
#    print(f"{exp_name}: {len(samples)}")
#    print(f"mean: {sum(samples)/len(samples)}")
#    print(f"std: {pd.Series(samples).std()}")
    


In [12]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

samples = list(exp_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(10, 450)


FriedmanchisquareResult(statistic=2962.141090909092, pvalue=0.0)

In [13]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[1.85247623e-75 1.85247623e-75 1.85247623e-75 1.85247623e-75
 1.85247623e-75 1.87738661e-75 1.88996676e-75 1.88996676e-75
 4.84275962e-74 8.27168845e-74 1.82538757e-73 3.17322456e-73
 2.21071360e-72 2.85257039e-72 3.49277097e-72 8.15323200e-72
 1.07276211e-69 5.43914463e-68 1.21499346e-66 1.34872141e-63
 1.17730124e-61 5.35040253e-61 1.34776644e-54 4.43662733e-54
 1.19036104e-52 1.01788310e-49 1.19437899e-48 1.85887911e-48
 1.73760925e-47 5.12115124e-45 1.12868356e-38 5.61502672e-33
 7.82010959e-33 1.05660457e-29 2.40009218e-28 9.31573573e-27
 8.50494002e-20 6.72277908e-19 1.43075656e-18 3.09300643e-15
 2.43941965e-06 8.74083281e-03 2.00259750e-02 5.95574063e-02
 8.76918880e-01]
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8773866077195533e-75 is st. significant
1.8899667609148573e-75 is st. significant
1.8899667609148573

(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False, False, False]),
 array([8.33614303e-74, 8.33614303e-74, 8.33614303e-74, 8.33614303e-74,
        8.33614303e-74, 8.33614303e-74, 8.33614303e-74, 8.33614303e-74,
        1.79182106e-72, 2.97780784e-72, 6.38885649e-72, 1.07889635e-71,
        7.29535488e-71, 9.12822525e-71, 1.08275900e-70, 2.44596960e-70,
        3.11101012e-68, 1.52296050e-66, 3.28048234e-65, 3.50667566e-62,
        2.94325309e-60, 1.28409661e-59, 3.09986281e-53, 9.76058012e-53,
        2.49975818e-51, 2.03576621e-48, 2.26932007e-47, 3.34598240e-47,
        2.95393573e-46, 8.19384199e-44, 1.69302534e-37, 7.86103741e-32,
        1.01661425e-31, 1.26792548e-28, 2.64010140e-27, 9.31573573e

In [14]:
diag_df = pd.DataFrame(exp_samples)
diag_df.columns

Index(['Lstm units=10 not model-rnn_cell_params-peephole_connections',
       'Lstm units=20 not model-rnn_cell_params-peephole_connections',
       'Lstm units=50 not model-rnn_cell_params-peephole_connections',
       'Lstm units=100 not model-rnn_cell_params-peephole_connections',
       'Lstm units=200 not model-rnn_cell_params-peephole_connections',
       'Lstm units=10 model-rnn_cell_params-peephole_connections',
       'Lstm units=20 model-rnn_cell_params-peephole_connections',
       'Lstm units=50 model-rnn_cell_params-peephole_connections',
       'Lstm units=100 model-rnn_cell_params-peephole_connections',
       'Lstm units=200 model-rnn_cell_params-peephole_connections'],
      dtype='object')

In [15]:
exp_samples.values()

dict_values([[1.6962585, 1.1091155, 1.6626619, 1.1102473, 3.388807, 2.072082, 1.109805, 2.2107937, 1.5751866, 1.2260565, 3.2697732, 3.3670433, 2.0335262, 2.197228, 2.2352352, 2.2592769, 1.3007413, 2.327385, 1.1308548, 1.3064936, 2.0728836, 1.2053356, 1.1134689, 1.2479621, 2.2093875, 4.1114564, 3.336509, 2.7443495, 3.2959027, 3.376367, 3.2462704, 1.1089163, 2.1035216, 1.2460898, 1.7311122, 3.2277095, 1.6953776, 1.2259754, 3.201282, 1.1807439, 3.792489, 1.1305166, 3.3914223, 2.5126824, 1.1149583, 0.9277396, 1.0776606, 1.333918, 1.6447055, 1.2207916, 1.1015456, 2.3635628, 1.3106674, 1.3131429, 2.4626505, 2.5348651, 2.3070493, 1.303742, 1.3062475, 1.4539431, 1.058996, 1.0875766, 1.3079493, 2.551346, 1.4103285, 1.626834, 1.2438822, 2.0780132, 0.8558728, 2.3449574, 2.0985796, 1.2206806, 1.4266354, 1.8505467, 1.4296181, 0.893399, 1.0743649, 0.8917019, 1.0041459, 1.0736036, 0.9397456, 1.0214362, 1.0588804, 1.068488, 1.2995807, 1.0845346, 1.3002893, 1.2858766, 1.3080248, 2.41251, 0.79175156, 1.

In [17]:
x=np.array(list(exp_samples.values()))
treatment_names = list(exp_samples.keys())
print(treatment_names)
x = x.T
diagram = Diagram(x,
    treatment_names=treatment_names,
    maximize_outcome = False,
)

diagram.to_file(
    "critdd_single_layer_lstm_peepholes_included.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

['Lstm units=10 not model-rnn_cell_params-peephole_connections', 'Lstm units=20 not model-rnn_cell_params-peephole_connections', 'Lstm units=50 not model-rnn_cell_params-peephole_connections', 'Lstm units=100 not model-rnn_cell_params-peephole_connections', 'Lstm units=200 not model-rnn_cell_params-peephole_connections', 'Lstm units=10 model-rnn_cell_params-peephole_connections', 'Lstm units=20 model-rnn_cell_params-peephole_connections', 'Lstm units=50 model-rnn_cell_params-peephole_connections', 'Lstm units=100 model-rnn_cell_params-peephole_connections', 'Lstm units=200 model-rnn_cell_params-peephole_connections']


In [29]:
def get_models_without_dropout(diagram: Diagram):
    """Gets the best optimizer for each type (Adam, RMSprop, SGD, ...)"""
    sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
    no_dropout_models = {}
    for name, avg_rank in sorted_treatments:
        if name.endswith("dropout=0.0"):
            #name = re.sub(" dropout=0.0", "", name)
            no_dropout_models[name] = (name, avg_rank)
    return no_dropout_models

def get_partial_diagram(subset_names, samples):
    x = np.array([samples[name] for name in subset_names])
    x = x.T
    return Diagram(x,
        treatment_names=subset_names,
        maximize_outcome = False,
    )

no_dropout_models = get_models_without_dropout(diagram)
partial_diagram = get_partial_diagram([name for name, _ in no_dropout_models.values()], exp_samples)

print("Previous average ranks:")
for name, avg_r in no_dropout_models.values():
    print(f"{name}: {avg_r}")

print("\nNew average ranks:")
sorted_treatments = sorted(zip(partial_diagram.treatment_names, partial_diagram.average_ranks), key=lambda x: x[1])
for name, avg_r in sorted_treatments:
    print(f"{name}: {avg_r}")

partial_diagram.to_file(
    "critdd_layernorm_no_drop.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

Previous average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd=0.2 dropout=0.0: 5.3933333333333335
Model-layer_normalization rd=0.05 dropout=0.0: 5.595555555555555
Model-layer_normalization rd=0.3 dropout=0.0: 6.0311111111111115
Not model-layer_normalization rd=0.3 dropout=0.0: 6.153333333333333
Not model-layer_normalization rd=0.1 dropout=0.0: 6.388888888888889
Not model-layer_normalization rd=0.05 dropout=0.0: 6.753333333333333

New average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd